In [ ]:
import pandas as pd
import numpy as np 
import string
import random

import nltk
from nltk.corpus import brown
from nltk.corpus import reuters
#nltk.download('reuters')

from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
import pickle

In [10]:
reuters.raw(fileids=['test/14832'])




"THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n  Thailand's trade deficit widened to 4.5\n  billion baht in the first quarter of 1987 from 2.1 billion a\n  year ago, the Business Economics Department said.\n      It said Janunary/March imports rose to 65.1 billion baht\n  from 58.7 billion. Thailand's improved business climate this\n  year resulted in a 27 pct increase in imports of raw materials\n  and semi-finished products.\n      The country's oil import bill, however, fell 23 pct in the\n  first quarter due to lower oil prices.\n      The department said first quarter exports expanded to 60.6\n  billion baht from 56.6 billion.\n      Export growth was smaller than expected due to lower\n  earnings from many key commodities including rice whose\n  earnings declined 18 pct, maize 66 pct, sugar 45 pct, tin 26\n  pct and canned pineapples seven pct.\n      Products registering high export growth were jewellery up\n  64 pct, clothing 57 pct and rubber 35 pct.\n  \n\n"

In [9]:
# remove punctuation from all DOCs 
punctuations = set(string.punctuation)
docslist = []
count=0
for index, i in  enumerate(reuters.fileids()):
    text = reuters.raw(fileids=[i])
    text = ''.join(ch for ch in text if ch not in punctuations)
    docslist.append(text)
    
print(docslist[2])

JAPAN TO REVISE LONGTERM ENERGY DEMAND DOWNWARDS
  The Ministry of International Trade and
  Industry MITI will revise its longterm energy supplydemand
  outlook by August to meet a forecast downtrend in Japanese
  energy demand ministry officials said
      MITI is expected to lower the projection for primary energy
  supplies in the year 2000 to 550 mln kilolitres kl from 600
  mln they said
      The decision follows the emergence of structural changes in
  Japanese industry following the rise in the value of the yen
  and a decline in domestic electric power demand
      MITI is planning to work out a revised energy supplydemand
  outlook through deliberations of committee meetings of the
  Agency of Natural Resources and Energy the officials said
      They said MITI will also review the breakdown of energy
  supply sources including oil nuclear coal and natural gas
      Nuclear energy provided the bulk of Japans electric power
  in the fiscal year ended March 31 supplying an est

In [ ]:
#tokenize words in all DOCS
tokenized_doc = []

for doc in docslist:
    tokentext = word_tokenize(doc)
    tokenized_doc.append(tokentext)
    
print(tokenized_doc[2])

In [ ]:
for x in range(len(reuters.fileids())):
    lowers = [word.lower() for word in tokenized_doc[x]]
    tokenized_doc[x] = lowers

tokenized_doc[2][0:10]

In [ ]:
# Creating inverse index which gives document number for each document and where word appears

#first we need to create a list of all words 

l = tokenized_doc
allwords = [item for sublist in l for item in sublist]
#print(allwords[0:100])
wordsunique = set(allwords)
wordsunique = list(wordsunique)
wordsunique[0:10]

def unique_words():
    l = tokenized_doc
    allwords = [item for sublist in l for item in sublist]
    #print(allwords[0:100])
    wordsunique = set(allwords)
    wordsunique = list(wordsunique)
    return wordsunique
    #wordsunique[0:10]




In [ ]:
# creating functions for TD-IDF / BM25
import math
from textblob import TextBlob as tb

def tf(word, doc):
    return doc.count(word) / len(doc)

def n_containing(word, doclist):
    return sum(1 for doc in doclist if word in doc)

def idf(word, doclist):
    return math.log(len(doclist) / (0.01 + n_containing(word, doclist)))

def tfidf(word, doc, doclist):
    return (tf(word, doc) * idf(word, doclist))

In [ ]:
# Creating dictonary of words
import re
import numpy as np
plottest = tokenized_doc[0:1000]

worddic = {}

for doc in plottest:
    for word in wordsunique:
        if word in doc:
            word = str(word)
            index = plottest.index(doc)
            positions = list(np.where(np.array(plottest[index]) == word)[0])
            idfs = tfidf(word,doc,plottest)
            try:
                worddic[word].append([index,positions,idfs])
            except:
                worddic[word] = []
                worddic[word].append([index,positions,idfs])

In [ ]:
np.array(plottest[0])=="japan"

In [ ]:
pickle_in = open("worddic_1000","rb")
worddic = pickle.load(pickle_in)

In [14]:
worddic['japan']

[[0, [14, 47, 249, 361, 378, 411, 435, 548, 600], 0.033386270829022735],
 [2, [0], 0.015288639114382281],
 [14,
  [0, 17, 41, 69, 88, 106, 167, 178, 216, 278, 285, 316, 444],
  0.07362772883992666],
 [27, [0, 13, 108], 0.040278191372408144],
 [28, [74], 0.026716510775637727],
 [32, [143], 0.0035125293051635254],
 [38, [30, 161], 0.025432063142193605],
 [42, [0, 13, 108], 0.040278191372408144],
 [43, [2, 11], 0.05943673183793561],
 [59, [29], 0.025432063142193605],
 [83, [266], 0.007266303754912459],
 [157, [0, 6], 0.1889238976277239],
 [169, [0, 188, 295, 302], 0.025867330726534327],
 [174, [16, 49], 0.05135795275316766],
 [202, [31, 74], 0.05037970603405972],
 [250, [84], 0.013029234319153373],
 [273, [286], 0.003163797328693941],
 [279, [330, 394], 0.008686156212768916],
 [282, [0, 20, 59, 82, 94, 113, 126, 153], 0.11315228093211271],
 [284, [5, 399], 0.008138260205501954],
 [285, [19, 163, 199], 0.027173985275220563],
 [299, [0, 36, 80, 124], 0.06151010620437523],
 [307, [10, 57], 0

In [3]:
# create word search which takes multiple words and finds documents that contain both along with metrics for ranking:

    ## (1) Number of occruances of search words 
    ## (2) TD-IDF score for search words 
    ## (3) Percentage of search terms
    ## (4) Word ordering score 
    ## (5) Exact match bonus 


from collections import Counter

def search(searchsentence):
    try:
        # split sentence into individual words 
        searchsentence = searchsentence.lower()
        try:
            words = searchsentence.split(' ')
        except:
            words = list(words)
        enddic = {}
        idfdic = {}
        closedic = {}
        
        # remove words if not in worddic 
        realwords = []
        for word in words:
            if word in list(worddic.keys()):
                realwords.append(word)  
        words = realwords
        numwords = len(words)
        
        # make metric of number of occurances of all words in each doc & largest total IDF 
        for word in words:
            for indpos in worddic[word]:
                index = indpos[0]
                amount = len(indpos[1])
                idfscore = indpos[2]
                #enddic[index] = amount
                try:
                    enddic[index].append(amount)
                except:
                    enddic[index] = []
                    enddic[index].append(amount)

                idfdic[index] = idfscore
                fullcount_order = sorted(enddic.items(), key=lambda x:np.sum(x[1]), reverse=True)
                fullidf_order = sorted(idfdic.items(), key=lambda x:np.sum(x[1]), reverse=True)
            #print(fullcount_order)
        # print(enddic)
        #print(fullcount_order)    


        #make metric of what percentage of words appear in each doc
        combo = []
        alloptions = {k: worddic.get(k, None) for k in (words)}
        #print(alloptions.values)

        for worddex in list(alloptions.values()):
            for indexpos in worddex:
                for indexz in indexpos:
                    combo.append(indexz)
        #print()
        comboindex = combo[::3]
        combocount = Counter(comboindex)
        
        for key in combocount:
            combocount[key] = combocount[key] / numwords
        combocount_order = sorted(combocount.items(), key=lambda x:x[1], reverse=True)
        
        # make metric for if words appear in same order as in search
        if len(words) > 1:
            x = []
            y = []
            for record in [worddic[z] for z in words]:
                for index in record:
                     x.append(index[0])
            print(x)
        #     for i in x:
        #         if x.count(i) > 1:
        #             y.append(i)
        #     y = list(set(y))

        #     closedic = {}
        #     for wordbig in [worddic[x] for x in words]:
        #         for record in wordbig:
        #             if record[0] in y:
        #                 index = record[0]
        #                 positions = record[1]
        #                 try:
        #                     closedic[index].append(positions)
        #                 except:
        #                     closedic[index] = []
        #                     closedic[index].append(positions)

        #     x = 0
        #     fdic = {}
        #     for index in y:
        #         csum = []
        #         for seqlist in closedic[index]:
        #             while x > 0:
        #                 secondlist = seqlist
        #                 x = 0
        #                 sol = [1 for i in firstlist if i + 1 in secondlist]
        #                 csum.append(sol)
        #                 fsum = [item for sublist in csum for item in sublist]
        #                 fsum = sum(fsum)
        #                 fdic[index] = fsum
        #                 fdic_order = sorted(fdic.items(), key=lambda x:x[1], reverse=True)
        #             while x == 0:
        #                 firstlist = seqlist
        #                 x = x + 1
        # else:
        #     fdic_order = 0
                    
        # # also the one above should be given a big boost if ALL found together 
           
        
        # #could make another metric for if they are not next to each other but still close 
        
        
        # return(searchsentence,words,fullcount_order,combocount_order,fullidf_order,fdic_order)
    
    except:
        return("")


search('indonesia crude palm oil')

''

In [1]:
if(os.path.exists('worddic_1000')):
    print("yes")
else:
    print("no")

yes
